# 스파크 고급 분석 4장

## 의사 결정 나무로 산림 식생 분포 예측하기

In [1]:
%%configure -f
{
    "proxyUser": "hduser",
    "driverMemory": "4000M", 
    "conf": {"spark.jars.packages": "graphframes:graphframes:0.3.0-spark2.0-s_2.11",
             "spark.master": "local[2]",
             "spark.sql.crossJoin.enabled": "true"}
}

In [2]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.classification.DecisionTreeClassifier
import scala.util.Random
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.tuning.TrainValidationSplit
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.VectorIndexer
import org.apache.spark.sql.DataFrame
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,None,spark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.classification.DecisionTreeClassifier
import scala.util.Random
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.tuning.TrainValidationSplit
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.VectorIndexer
import org.apache.spark.sql.DataFrame
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator


In [2]:
val base = "hdfs://localhost:54310/covtype/"


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,None,spark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

base: String = hdfs://localhost:54310/covtype/


## 학습/평가 데이터 준비

In [3]:
val dataWithoutHeader = spark.read.option("inferSchema", "true").option("header", false).csv(base + "covtype.data")
val colNames = Seq(
        "Elevation", "Aspect", "Slope",
        "Horizontal_Distance_To_Hydrology", "Vertical_Distance_To_Hydrology",
        "Horizontal_Distance_To_Roadways",
        "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm",
        "Horizontal_Distance_To_Fire_Points"
      ) ++ (
        (0 until 4).map(i => s"Wilderness_Area_$i")
      ) ++ (
        (0 until 40).map(i => s"Soil_Type_$i")
      ) ++ Seq("Cover_Type")
val data = dataWithoutHeader.toDF(colNames: _*).withColumn("Cover_Type", $"Cover_Type".cast("double"))
val Array(trainData, testData) = data.randomSplit(Array(0.9, 0.1))
trainData.cache()
testData.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dataWithoutHeader: org.apache.spark.sql.DataFrame = [_c0: int, _c1: int ... 53 more fields]
colNames: Seq[String] = List(Elevation, Aspect, Slope, Horizontal_Distance_To_Hydrology, Vertical_Distance_To_Hydrology, Horizontal_Distance_To_Roadways, Hillshade_9am, Hillshade_Noon, Hillshade_3pm, Horizontal_Distance_To_Fire_Points, Wilderness_Area_0, Wilderness_Area_1, Wilderness_Area_2, Wilderness_Area_3, Soil_Type_0, Soil_Type_1, Soil_Type_2, Soil_Type_3, Soil_Type_4, Soil_Type_5, Soil_Type_6, Soil_Type_7, Soil_Type_8, Soil_Type_9, Soil_Type_10, Soil_Type_11, Soil_Type_12, Soil_Type_13, Soil_Type_14, Soil_Type_15, Soil_Type_16, Soil_Type_17, Soil_Type_18, Soil_Type_19, Soil_Type_20, Soil_Type_21, Soil_Type_22, Soil_Type_23, Soil_Type_24, Soil_Type_25, Soil_Type_26, Soil_Type_27, Soil_Type_28, Soil_Type_29, Soil_Type_30, Soil_Type_31, Soil_Type_32, Soil_Type_33, Soil_Type_34, Soil_...data: org.apache.spark.sql.DataFrame = [Elevation: int, Aspect: int ... 53 more fields]
trainData: org.apach

In [4]:
trainData.head

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res3: org.apache.spark.sql.Row = [1859,18,12,67,11,90,211,215,139,792,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0]


## 벡터화

In [11]:
val inputCols = trainData.columns.filterNot( _ == "Cover_Type")
val assembler = new VectorAssembler().
    setInputCols(inputCols).
    setOutputCol("featureVector")

val assembledTrainData = assembler.transform(trainData)
assembledTrainData.select("featureVector").show(truncate=false)



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.ml.feature.VectorAssembler
inputCols: Array[String] = Array(Elevation, Aspect, Slope, Horizontal_Distance_To_Hydrology, Vertical_Distance_To_Hydrology, Horizontal_Distance_To_Roadways, Hillshade_9am, Hillshade_Noon, Hillshade_3pm, Horizontal_Distance_To_Fire_Points, Wilderness_Area_0, Wilderness_Area_1, Wilderness_Area_2, Wilderness_Area_3, Soil_Type_0, Soil_Type_1, Soil_Type_2, Soil_Type_3, Soil_Type_4, Soil_Type_5, Soil_Type_6, Soil_Type_7, Soil_Type_8, Soil_Type_9, Soil_Type_10, Soil_Type_11, Soil_Type_12, Soil_Type_13, Soil_Type_14, Soil_Type_15, Soil_Type_16, Soil_Type_17, Soil_Type_18, Soil_Type_19, Soil_Type_20, Soil_Type_21, Soil_Type_22, Soil_Type_23, Soil_Type_24, Soil_Type_25, Soil_Type_26, Soil_Type_27, Soil_Type_28, Soil_Type_29, Soil_Type_30, Soil_Type_31, Soil_Type_32, Soil_Type_33, Soil_Type_34, S...assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_7822c2cfc3e7
assembledTrainData: org.apache.spark.sql.DataFrame = [Elevation: i

## 트리 구축 및 학습

In [14]:
val model = new DecisionTreeClassifier().
    setLabelCol("Cover_Type").
    setFeaturesCol("featureVector").
    setPredictionCol("prediction").
    fit(assembledTrainData)
println(model.toDebugString)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.ml.classification.DecisionTreeClassifier
import scala.util.Random
model: org.apache.spark.ml.classification.DecisionTreeClassificationModel = DecisionTreeClassificationModel (uid=dtc_b91359e8f0e8) of depth 5 with 41 nodes
DecisionTreeClassificationModel (uid=dtc_b91359e8f0e8) of depth 5 with 41 nodes
  If (feature 0 <= 3048.5)
   If (feature 0 <= 2476.5)
    If (feature 3 <= 15.0)
     If (feature 12 <= 0.5)
      If (feature 23 <= 0.5)
       Predict: 4.0
      Else (feature 23 > 0.5)
       Predict: 3.0
     Else (feature 12 > 0.5)
      Predict: 6.0
    Else (feature 3 > 15.0)
     If (feature 16 <= 0.5)
      Predict: 3.0
     Else (feature 16 > 0.5)
      If (feature 9 <= 1288.5)
       Predict: 3.0
      Else (feature 9 > 1288.5)
       Predict: 4.0
   Else (feature 0 > 2476.5)
    If (feature 17 <= 0.5)
     If (feature 15 <= 0.5)
      Predict: 2.0
     Else (feature 15 > 0.5)
      Predict: 3.0
    Else (feature 17 > 0.5)
     If (feature 0 <= 2697.5)
 

In [19]:
model.featureImportances.toArray.zip(inputCols).sorted.reverse.foreach(println)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(0.8243706121415982,Elevation)
(0.04250140940349825,Soil_Type_3)
(0.03314313258774212,Soil_Type_1)
(0.02693294405496738,Hillshade_Noon)
(0.024444469339088008,Horizontal_Distance_To_Hydrology)
(0.018481412770745796,Soil_Type_31)
(0.015692478668951324,Wilderness_Area_2)
(0.004601426844001658,Horizontal_Distance_To_Roadways)
(0.0037830258112822664,Soil_Type_2)
(0.002809083504044355,Hillshade_9am)
(0.00227764710278306,Horizontal_Distance_To_Fire_Points)
(9.623577712976215E-4,Soil_Type_9)
(0.0,Wilderness_Area_3)
(0.0,Wilderness_Area_1)
(0.0,Wilderness_Area_0)
(0.0,Vertical_Distance_To_Hydrology)
(0.0,Soil_Type_8)
(0.0,Soil_Type_7)
(0.0,Soil_Type_6)
(0.0,Soil_Type_5)
(0.0,Soil_Type_4)
(0.0,Soil_Type_39)
(0.0,Soil_Type_38)
(0.0,Soil_Type_37)
(0.0,Soil_Type_36)
(0.0,Soil_Type_35)
(0.0,Soil_Type_34)
(0.0,Soil_Type_33)
(0.0,Soil_Type_32)
(0.0,Soil_Type_30)
(0.0,Soil_Type_29)
(0.0,Soil_Type_28)
(0.0,Soil_Type_27)
(0.0,Soil_Type_26)
(0.0,Soil_Type_25)
(0.0,Soil_Type_24)
(0.0,Soil_Type_23)
(0.0,Soi

## 예측 수행

In [25]:
// 학습 데이터의 예측
val predictions = model.transform(assembledTrainData)
predictions.select("Cover_Type", "prediction", "probability").show(truncate=false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

predictions: org.apache.spark.sql.DataFrame = [Elevation: int, Aspect: int ... 57 more fields]
+----------+----------+-------------------------------------------------------------------------------------------------------------------+
|Cover_Type|prediction|probability                                                                                                        |
+----------+----------+-------------------------------------------------------------------------------------------------------------------+
|3.0       |3.0       |[0.0,3.4258307639602604E-5,0.048304213771839674,0.6236382322713258,0.02195957519698527,0.0,0.30606372045220964,0.0]|
|3.0       |3.0       |[0.0,3.4258307639602604E-5,0.048304213771839674,0.6236382322713258,0.02195957519698527,0.0,0.30606372045220964,0.0]|
|3.0       |3.0       |[0.0,3.4258307639602604E-5,0.048304213771839674,0.6236382322713258,0.02195957519698527,0.0,0.30606372045220964,0.0]|
|6.0       |3.0       |[0.0,3.4258307639602604E-5,0.0483042137718

## 평가

In [30]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

val evaluator = new MulticlassClassificationEvaluator().
    setLabelCol("Cover_Type").
    setPredictionCol("prediction")

evaluator.setMetricName("accuracy").evaluate(predictions)
evaluator.setMetricName("f1").evaluate(predictions)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_d5da4cd87dd8
res42: Double = 0.7033956516830929
res43: Double = 0.6879709790220652


### 혼동 행렬 계산

In [47]:
import org.apache.spark.mllib.evaluation.MulticlassMetrics

// MulticlassMetrics
//   param: predictionAndLabels an RDD of (prediction, label, weight, probability) or (prediction, label, weight) or (prediction, label) tuples.

// val predictionRDD = predictions.
//     select("prediction", "Cover_Type").
//     as[(Double, Double)].
//     rdd

// val multiClassMetrics = new MulticlassMetrics(predictionRDD)
// multiClassMetrics.confusionMatrix

// 직접 구현 (by Pivot)

val confusionMatrix = predictions.
    groupBy("Cover_Type").
    pivot("prediction", (1 to 7)).
    count.
    na.fill(0.0).
    orderBy("Cover_Type")
confusionMatrix.show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.mllib.evaluation.MulticlassMetrics
confusionMatrix: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Cover_Type: double, 1: bigint ... 6 more fields]
+----------+------+------+-----+----+---+---+-----+
|Cover_Type|1     |2     |3    |4   |5  |6  |7    |
+----------+------+------+-----+----+---+---+-----+
|1.0       |128661|56251 |83   |0   |34 |3  |5317 |
|2.0       |51510 |199538|2785 |33  |358|37 |782  |
|3.0       |0     |4665  |27089|357 |35 |93 |0    |
|4.0       |0     |12    |1325 |1134|0  |0  |0    |
|5.0       |0     |7827  |260  |0   |432|0  |0    |
|6.0       |0     |5049  |9948 |132 |11 |401|0    |
|7.0       |8031  |79    |0    |0   |0  |0  |10367|
+----------+------+------+-----+----+---+---+-----+



## 70% 정확도가 괜찮은 것일까? 

무작위 분류기의 성능보다 좋은지
* 학습 class prior * 평가 class prior 가 각 example을 맞출(accurate) 확률 
* 이를 누적하여 무작위 분류기의 accuracy 추계

결론 : 70% 가 37% 보다는 우수하므로 적어도 무작위 분류기보다는 성능이 우수하다

In [53]:
def getClassPrior(df:DataFrame): Array[Double] = {
    val totalCount = df.count
    
    df.groupBy("Cover_Type").count.
       orderBy("Cover_Type").
       select("count").as[Double].
       map ( _ / totalCount).
       collect()
}

val trainDataClassProbs = getClassPrior(trainData)
val testDataClassProbs = getClassPrior(testData)

trainDataClassProbs.zip(testDataClassProbs).map { case (trainClassProb, testClassProb) =>
        trainClassProb * testClassProb
}.sum

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.DataFrame
getClassPrior: (df: org.apache.spark.sql.DataFrame)Array[Double]
trainDataClassProbs: Array[Double] = Array(0.364207416591567, 0.4879907546126485, 0.06168502541907512, 0.004727928838069872, 0.016299969960144572, 0.029735630138585142, 0.035353274439909765)
testDataClassProbs: Array[Double] = Array(0.36816678944032344, 0.4840936734449146, 0.06021619584396896, 0.00472821338632587, 0.01668579651551231, 0.03128158566460521, 0.034827745704349614)
res153: Double = 0.3764925349876059


## 의사 결정 나무 HPO

* 참고 : https://spark.apache.org/docs/latest/ml-tuning.html

* By TrainValidationSplit 
  * estimator(pipeline)
  * evaluator
  * paramGrid
  * fold = 1
* pram search 
  * 불순도 기준, 최대 깊이, 최대Bin개수, 최소 infoGain

In [5]:

// 파이프라인 구축: assembler, classifier
val inputCols = trainData.columns.filterNot( _ == "Cover_Type")
val assembler = new VectorAssembler().
    setInputCols(inputCols).
    setOutputCol("featureVector")
val classifier = new DecisionTreeClassifier().
    setSeed(Random.nextLong()).
    setLabelCol("Cover_Type").
    setFeaturesCol("featureVector").
    setPredictionCol("prediction")
val pipeline = new Pipeline().setStages(Array(assembler, classifier))

// evaluator 준비
val evaluator = new MulticlassClassificationEvaluator().
    setLabelCol("Cover_Type").
    setPredictionCol("prediction").
    setMetricName("accuracy")

// ParamGrid 준비
val paramGrid = new ParamGridBuilder().
    addGrid(classifier.impurity, Seq("gini", "entropy")).
    addGrid(classifier.maxDepth, Seq(1, 20)).
    addGrid(classifier.maxBins, Seq(40, 300)).
    addGrid(classifier.minInfoGain, Seq(0.0, 0.05)).
    build()


val validator = new TrainValidationSplit().
    setSeed(Random.nextLong()).
    setEstimator(pipeline).
    setEvaluator(evaluator).
    setEstimatorParamMaps(paramGrid).
    setTrainRatio(0.9)

val validatorModel = validator.fit(trainData)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.classification.DecisionTreeClassifier
import scala.util.Random
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.tuning.TrainValidationSplit
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
inputCols: Array[String] = Array(Elevation, Aspect, Slope, Horizontal_Distance_To_Hydrology, Vertical_Distance_To_Hydrology, Horizontal_Distance_To_Roadways, Hillshade_9am, Hillshade_Noon, Hillshade_3pm, Horizontal_Distance_To_Fire_Points, Wilderness_Area_0, Wilderness_Area_1, Wilderness_Area_2, Wilderness_Area_3, Soil_Type_0, Soil_Type_1, Soil_Type_2, Soil_Type_3, Soil_Type_4, Soil_Type_5, Soil_Type_6, Soil_Type_7, Soil_Type_8, Soil_Type_9, Soil_Type_10, Soil_Type_11, Soil_Type_12, Soil_Type_13, Soil_Type_14, Soil_Type_15, Soil_Type_16, Soil_Type_17, Soil_Type_18, Soil_Type_19, Soil_Type_20, Soil_Type_21, Soil_Type_22, 

In [10]:
validatorModel.bestModel.asInstanceOf[PipelineModel].stages.last.extractParamMap   // BEST 모델의 parameter

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res22: org.apache.spark.ml.param.ParamMap =
{
	dtc_624f0acc2372-cacheNodeIds: false,
	dtc_624f0acc2372-checkpointInterval: 10,
	dtc_624f0acc2372-featuresCol: featureVector,
	dtc_624f0acc2372-impurity: entropy,
	dtc_624f0acc2372-labelCol: Cover_Type,
	dtc_624f0acc2372-maxBins: 300,
	dtc_624f0acc2372-maxDepth: 20,
	dtc_624f0acc2372-maxMemoryInMB: 256,
	dtc_624f0acc2372-minInfoGain: 0.0,
	dtc_624f0acc2372-minInstancesPerNode: 1,
	dtc_624f0acc2372-predictionCol: prediction,
	dtc_624f0acc2372-probabilityCol: probability,
	dtc_624f0acc2372-rawPredictionCol: rawPrediction,
	dtc_624f0acc2372-seed: 6205718850843740355
}


In [18]:
// 각 조합별 파라미터 
validatorModel.validationMetrics.zip(validatorModel.getEstimatorParamMaps).sortBy(_._1).reverse.foreach {
    case (metric, paramMap) =>
    println(metric)
    println(paramMap)
    println()
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.9092202125819286
{
	dtc_624f0acc2372-impurity: entropy,
	dtc_624f0acc2372-maxBins: 300,
	dtc_624f0acc2372-maxDepth: 20,
	dtc_624f0acc2372-minInfoGain: 0.0
}

0.9080285236511811
{
	dtc_624f0acc2372-impurity: entropy,
	dtc_624f0acc2372-maxBins: 40,
	dtc_624f0acc2372-maxDepth: 20,
	dtc_624f0acc2372-minInfoGain: 0.0
}

0.90545293789763
{
	dtc_624f0acc2372-impurity: gini,
	dtc_624f0acc2372-maxBins: 300,
	dtc_624f0acc2372-maxDepth: 20,
	dtc_624f0acc2372-minInfoGain: 0.0
}

0.9045495608049666
{
	dtc_624f0acc2372-impurity: gini,
	dtc_624f0acc2372-maxBins: 40,
	dtc_624f0acc2372-maxDepth: 20,
	dtc_624f0acc2372-minInfoGain: 0.0
}

0.7271224556480289
{
	dtc_624f0acc2372-impurity: entropy,
	dtc_624f0acc2372-maxBins: 40,
	dtc_624f0acc2372-maxDepth: 20,
	dtc_624f0acc2372-minInfoGain: 0.05
}

0.7254118054087301
{
	dtc_624f0acc2372-impurity: entropy,
	dtc_624f0acc2372-maxBins: 300,
	dtc_624f0acc2372-maxDepth: 20,
	dtc_624f0acc2372-minInfoGain: 0.05
}

0.6709016472216349
{
	dtc_624f0acc2372-impurity: 

## 범주형 변수를 one-hot에서 numerical encoding을 변경 / Tree에 알려주기 /HPO 다시 


In [29]:
// 데이터 변환 -> 분류 -> 평가 

def unencodeOneHot(data:DataFrame): DataFrame = {
    
    val wildernessCols = (0 until 4).map( i => s"Wilderness_Area_$i" ).toArray
        
    val wildernessAssembler = new VectorAssembler().
        setInputCols(wildernessCols).
        setOutputCol("wilderness")
    
    val unhotUDF = udf { (vec:Vector) => vec.toArray.indexOf(1) }
    
    val withWilderness = wildernessAssembler.transform(data).
        drop(wildernessCols: _*).
        withColumn("wilderness", unhotUDF($"wilderness"))
    
    val soilCols = (0 until 40).map( i => s"Soil_Type_$i" ).toArray
    
    val soilAssembler = new VectorAssembler().
        setInputCols(soilCols).
        setOutputCol("soil")
    
    soilAssembler.transform(withWilderness).
        drop(soilCols: _*).
        withColumn("soil", unhotUDF($"soil"))
}

val unencTrainData = unencodeOneHot(trainData)
val unencTestData = unencodeOneHot(testData) 

val inputCols = unencTrainData.columns.filterNot( _ == "Cover_Type")
val assembler = new VectorAssembler().
    setInputCols(inputCols).
    setOutputCol("featureVector")

val indexer = new VectorIndexer().
    setInputCol("featureVector").
    setOutputCol("indexedVector").
    setMaxCategories(40)

// FOR DEBUG
// val indexerModel = indexer.fit(assembler.transform(unencTrainData))
// val categoricalFeatures = indexerModel.categoryMaps.keys.toSet
// println(s"Chose ${categoricalFeatures.size} " +
//         s"categorical features: ${categoricalFeatures.mkString(", ")}")


val classifier = new DecisionTreeClassifier().
    setSeed(Random.nextLong()).
    setLabelCol("Cover_Type").
    setFeaturesCol("indexedVector").
    setPredictionCol("prediction")

val pipeline = new Pipeline().setStages(Array(assembler, indexer, classifier))

val paramGrid = new ParamGridBuilder().
    addGrid(classifier.impurity, Seq("gini", "entropy")).
    addGrid(classifier.maxDepth, Seq(1, 20)).
    addGrid(classifier.maxBins, Seq(40, 300)).
    addGrid(classifier.minInfoGain, Seq(0.0, 0.05)).
    build()

val multiclassEval = new MulticlassClassificationEvaluator().
    setLabelCol("Cover_Type").
    setPredictionCol("prediction").
    setMetricName("accuracy")

val validator = new TrainValidationSplit().
    setSeed(Random.nextLong()).
    setEstimator(pipeline).
    setEvaluator(multiclassEval).
    setEstimatorParamMaps(paramGrid).
    setTrainRatio(0.9)

val validatorModel = validator.fit(unencTrainData)

val bestModel = validatorModel.bestModel

println(bestModel.asInstanceOf[PipelineModel].stages.last.extractParamMap)

val testAccuracy = multiclassEval.evaluate(bestModel.transform(unencTestData))
println(testAccuracy)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

unencodeOneHot: (data: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame
unencodedTrainData: org.apache.spark.sql.DataFrame = [Elevation: int, Aspect: int ... 11 more fields]
inputCols: Array[String] = Array(Elevation, Aspect, Slope, Horizontal_Distance_To_Hydrology, Vertical_Distance_To_Hydrology, Horizontal_Distance_To_Roadways, Hillshade_9am, Hillshade_Noon, Hillshade_3pm, Horizontal_Distance_To_Fire_Points, wilderness, soil)
assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_2bed32fa9733
indexer: org.apache.spark.ml.feature.VectorIndexer = vecIdx_50aeb0916000
indexerModel: org.apache.spark.ml.feature.VectorIndexerModel = vecIdx_50aeb0916000
categoricalFeatures: scala.collection.immutable.Set[Int] = Set(10, 11)
Chose 2 categorical features: 10, 11
classifier: org.apache.spark.ml.classification.DecisionTreeClassifier = dtc_82f67bc881a1
pipeline: org.apache.spark.ml.Pipeline = pipeline_7537e0d648e8
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
A

# 랜덤 포레스트

In [ ]:
import org.apache.spark.ml.classification.RandomForestClassifier


def evaluateForest() = {
    val unencTrainData = unencodeOneHot(trainData)
    val unencTestData = unencodeOneHot(testData)    

    val inputCols = unencTrainData.columns.filterNot( _ == "Cover_Type")
    val assembler = new VectorAssembler().
        setInputCols(inputCols).
        setOutputCol("featureVector")

    val indexer = new VectorIndexer().
        setInputCol("featureVector").
        setOutputCol("indexedVector").
        setMaxCategories(40)

    val classifier = new RandomForestClassifier().
        setSeed(Random.nextLong()).
        setLabelCol("Cover_Type").
        setFeaturesCol("indexedVector").
        setPredictionCol("prediction")    

    val pipeline = new Pipeline().setStages(Array(assembler, indexer, classifier))
    
    val paramGrid = new ParamGridBuilder().
        addGrid(classifier.impurity, Seq("entropy")).
        addGrid(classifier.maxDepth, Seq(20)).
        addGrid(classifier.maxBins, Seq(300)).
        addGrid(classifier.minInfoGain, Seq(0.0)).
        build()

    val multiclassEval = new MulticlassClassificationEvaluator().
        setLabelCol("Cover_Type").
        setPredictionCol("prediction").
        setMetricName("accuracy")

    val validator = new TrainValidationSplit().
        setSeed(Random.nextLong()).
        setEstimator(pipeline).
        setEvaluator(multiclassEval).
        setEstimatorParamMaps(paramGrid).
        setTrainRatio(0.9)

    val validatorModel = validator.fit(unencTrainData)

    val bestModel = validatorModel.bestModel

    println(bestModel.asInstanceOf[PipelineModel].stages.last.extractParamMap)

    val testAccuracy = multiclassEval.evaluate(bestModel.transform(unencTestData))
    println(testAccuracy)    
}

evaluateForest()



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…